In [6]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import torchvision.models as models
import torch.nn as nn


In [13]:
import torch
import torch.nn as nn
import torchvision.models as models

# Load model dengan pretrained weights
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)

# Periksa struktur classifier
print(model.classifier)

# Modifikasi classifier sesuai dengan model yang sudah dilatih
model.classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(model.last_channel, 2)  # Sesuai jumlah kelas (worn / not worn)
)

checkpoint = torch.load("best_model.h5", map_location="cpu")

# Rename key dari "classifier.1.1" ke "classifier.1"
new_state_dict = {}
for key in checkpoint:
    new_key = key.replace("classifier.1.1", "classifier.1")  # Perbaiki key yang salah
    new_state_dict[new_key] = checkpoint[key]

# Load ulang dengan model yang sudah diperbaiki
model.load_state_dict(new_state_dict)
model = model.to(device)
model.eval()

print("Model berhasil dimuat!")

Sequential(
  (0): Dropout(p=0.2, inplace=False)
  (1): Linear(in_features=1280, out_features=1000, bias=True)
)
Model berhasil dimuat!


In [14]:
# Transformasi gambar agar sesuai dengan model (misalnya, ukuran 224x224 untuk MobileNetV2)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalisasi sesuai ImageNet
])

def load_image(image_path):
    """Load dan preproses gambar"""
    image = Image.open(image_path).convert("RGB")  # Buka gambar
    image = transform(image)  # Terapkan transformasi
    image = image.unsqueeze(0)  # Tambahkan batch dimension
    return image.to(device)


In [16]:
def predict(image_path):
    """Fungsi untuk melakukan prediksi pada gambar baru"""
    image = load_image(image_path)
    
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
    
    class_names = ["Memakai-Helm", "Tidak-Memakai-Helm"]  # Sesuai urutan pelatihan
    print(f"Prediksi: {class_names[predicted.item()]}")
    
    return class_names[predicted.item()]

# Contoh penggunaan
image_path = "C:\\Users\\User\\Downloads\\helmet-story_647_010216125130_062417031850.jpg"  # Ganti dengan path gambar yang ingin diuji"
prediction = predict(image_path)

Prediksi: Tidak-Memakai-Helm
